# Graphene Size Optimization

Graphene blocks are comprised of two major data structures, an Invertible Bloom Lookup Table (IBLT) labeled $I$ and a Bloom filter (BF) labeled $B$.
In this document we analyze two size tuning procedures for Graphene blocks. Although the [original Graphene paper](https://people.cs.umass.edu/~gbiss/graphene.pdf) offered an optimization formula, the current version (as of commit 786c1d1) uses brute-force search to find optimal sizes for $I$ and $B$. This decision was made because the optimization formula is unable to account for discontinuous functions such as floors and ceilings. As a result, the formula is inaccurate, particularly for Graphene blocks containing relatively few transactions. Nevertheless, for large blocks, the formula should work quite well, and is much more efficient than brute-force search. Therefore, one of the goals of this document is to determine how large a block must be in order for the optimization formula to work well.

The second goal of this document is to determine a suitable increase in the size of $I$ that can compensate for the natural variance in the false positive rate (FPR) of $B$. It is relatively easy to determine the expected number of false positives (FPs) for a BF of given size and holding a given number of items. However, some fraction of the time, more FPs will be generated than expected. In such cases we want $I$ to be large enough to decode.

## Optimization formula

There are two sources of error that must be accounted for when sizing $I$ and $B$: 

1. Transactions $M$ that are in the block but missing from the receiver mempool. 
2. Transactions $S$ that are not the block but are in the receiver mempool and which pass through the $B$ as FPs. 

$I$ must be sized large enough so that all transactions in $M \cup S$ can be recovered. Define $E$ to be the set of *excess transactions* that are in the receiver mempool, but not in the block. If $f_B$ is the FPR for $B$, then in expectation, $f_B |E| = |S|$. It is not possible for the sender to control $M$, but the size of $S$ is completely in his control. By reducing the FPR of $B$, the sender can make $S$ arbitrarily small, reducing the required size of $I$. However reducing the FPR of $B$ also requires an increase in the size of $B$. 

Thus, optimizing Graphene block size can be accomplished by determining the ideal choice for $s = |S|$, which leads to a minimal combined size for $I$ and $B$. Working with the actual sizes for $I$ and $B$, that arise from their c++ implementations, along with a simplified definition of FPR $f_B$ that ignores floors and ceilings, we can develop an expression for total size $T$. We begin with a table of variables.

| variable | description |
|:--------:|-------------|
| $b$ | BF cell size (bytes)|
| $n$ | items in block |
| $i$ | IBLT cell size (bytes) |
| $v$ | IBLT overhead |
| $M$ | Missing transactions as defined above |
| $S$ | False positives as defined above |
| $E$ | Excess transactions in mempool |
| $s$ | size of $S$, $|S|$
| $f_B(s)$ | implied FPR of $B$ given $s$ | 
| $I(s)$ | Size of $I$ given $s$ |
| $B(s)$ | Size $B$ given $s$ |
| $T(s)$ | Aggregate size of $I$ and $B$ given s| 

Based on the definitions above, we have.

$$f_B(s) = \frac{s}{|E|}$$
$$B(s) = \frac{b n}{8} \left( \frac{-1}{\ln^2(2)} \right) \ln(f_B(s))$$
$$I(s) = i v s$$
$$T(s) =  I(s) + B(s)$$

Taking the derivative of $T(s)$ with respect to $s$ and setting the result equal to zero, we can solve for the optimal value of $s$:

$$s^* = \frac{bn}{8iv \ln^2(2)}.$$